In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib
import matplotlib.pyplot as plt

import generate as generate
from single_layer import *
from smart_train import *

In [28]:
# Constants
num = 1
N_test = 102400
noise = 0.1

In [31]:
T = 256
M = 1
d = 8
(thetan, an, bn) = generate.generate_single_layer_v2(M, d, num)

(X, Y_noiseless) = generate.generate_single_data_v2(T, an, bn, thetan)
Y = generate.add_noise(Y_noiseless, noise)
input = X[0]
print(X.shape, Y.shape)
print(np.mean(Y))
print(np.std(Y))

(1, 256, 8) (1, 256)
0.17094281245960552
0.5315332094199331


In [38]:
hyperParamOpt = hyper_param_search(
    X[0], Y[0], (2, 2 * np.sqrt(T)), maxiter=100,
    val_ratio=0.2, 
    lr=0.002, 
    weight_decay=0.0000,
    batchNorm=False,
    dropout=0,
    batch_size=64,
    patience=20, 
    epochs=1000,
    verbose=False,
)
(X_test, Y_test) = generate.generate_single_data_v2(N_test, an, bn, thetan)
predicted = hyperParamOpt.model(torch.Tensor(X_test[0])).detach().numpy()
kl_divergence = generate.kl_divergence(predicted.reshape(-1), Y_test.reshape(-1), noise)
print(kl_divergence)

2 0.03135613 0.014004566241055727
32 0.08356825 0.03037984576076269
2 0.034203123 0.010593470884487033
5 0.056833632 0.008512852247804403
2 0.035558257 0.012542514130473137
2 0.070331246 0.0691053681075573
2 0.038835146 0.015106091275811195
(1.0, array(0.03420312, dtype=float32), 12)
best_loss: 0.03135612979531288, model: MultiLayer(
  (fcs): ModuleList(
    (0): Linear(in_features=8, out_features=2, bias=True)
    (1): Linear(in_features=2, out_features=2, bias=True)
    (2): Linear(in_features=2, out_features=2, bias=True)
    (3): Linear(in_features=2, out_features=1, bias=True)
  )
  (dropout): Dropout(p=0, inplace=False)
  (act): LeakyReLU(negative_slope=0.01)
), best_hidden_dim: 2, epoch_number: 217, fixed_params: {'val_ratio': 0.2, 'lr': 0.002, 'weight_decay': 0.0, 'batchNorm': False, 'dropout': 0, 'batch_size': 64, 'patience': 20, 'epochs': 1000, 'verbose': False}
0.7249050826163702
